In [1]:
#IMPORTS
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV #splits up your test set in eqally sized parts, uses one part as test data and the rest as training data. So it optimizes as many classifiers as parts you split your data into.


In [2]:
#LOAD DATA
df = pd.read_csv('USER-FEATURE-DATA.csv')
y = df.iloc[:, 0]
X = df.iloc[:, 1:]
print(y[5]) #Can't directly reference the index, must do this weird 0 indexing 

user1


In [25]:
#TRAINING
kf = KFold(n_splits=5)

ATTRIBUTES = len(X.columns)
print('n attrb =', ATTRIBUTES)
USERS = y.nunique()
print('n users =', USERS)
neurons = (ATTRIBUTES + USERS) // 2 
clf = MLPClassifier(hidden_layer_sizes=(neurons,), solver='lbfgs', momentum=0.2, learning_rate_init=0.3, random_state=0)
clf.n_layers_ = 3
clf.max_iter = 1000
#got rid of:
#hidden_layer_sizes=(1,) (scores all zeroes)
#  l
#
# look into RandomSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd'],
            'hidden_layer_sizes': [
              (neurons-10,), (neurons-5,), (neurons,), (neurons+5,), (neurons+10,)
             ],
             'momentum': [0.1,0.2,0.3],
             'learning_rate_init':[0.2,0.3,0.4]
        }
       ]

GRID_SEARCH_TEST = False 

for train_index, test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]

    if GRID_SEARCH_TEST:
        clf = GridSearchCV(MLPClassifier(), param_grid, cv=3, scoring='accuracy')
        clf.fit(X_train, y_train)
        print('params:', clf.best_params_)
    else:
      clf.fit(X_train, y_train)
      print('SCORE:', clf.score(X_test, y_test))
    """ 
    EXTRA INFO:
    # 

      print('current loss computed with the loss function: ',clf.loss_)
      #print('coefs: ', clf.coefs_)
      print('intercepts: ',clf.intercepts_)
      print(' number of iterations the solver: ', clf.n_iter_)
      print('num of layers: ', clf.n_layers_)
      print('Num of o/p: ', clf.n_outputs_) 
      
      """



n attrb = 102
n users = 20
SCORE: 0.058823529411764705
SCORE: 0.13725490196078433
SCORE: 0.2549019607843137
SCORE: 0.27450980392156865
SCORE: 0.04
